In [3]:
with open('shakespeare.txt','r',encoding='utf-8') as f:
    text = f.read()

In [4]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [5]:
len(text)

1115394

In [6]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


In [7]:
stoi = {ch : i for i,ch in enumerate(chars)}
itos = {i:ch for ch,i in stoi.items()}
encode = lambda x : [stoi[c] for c in x]
decode = lambda x : [itos[c] for c in x]

print(encode("Hello World"))
print(decode(encode("Hello World")))

[20, 43, 50, 50, 53, 1, 35, 53, 56, 50, 42]
['H', 'e', 'l', 'l', 'o', ' ', 'W', 'o', 'r', 'l', 'd']


In [40]:
import torch
import torch.nn as nn
from torch.nn import functional as F


In [9]:
data = torch.tensor(encode(text),dtype=torch.long)
data.shape

torch.Size([1115394])

In [10]:
data[:500]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
        47, 59, 57,  1, 47, 57,  1, 41, 

In [22]:
split = int(len(data) * 0.9)
train_data = data[:split]
test_data = data[split:]

In [23]:
block_size = 8
train_data[:block_size + 1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [36]:
torch.manual_seed(42)
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split == "train" else test_data
    ix = torch.randint(len(data)-block_size,(batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x,y



In [37]:
X.shape,y.shape

(torch.Size([4, 8]), torch.Size([4, 8]))

In [38]:
X

tensor([[57,  1, 46, 47, 57,  1, 50, 53],
        [ 1, 58, 46, 43, 56, 43,  1, 41],
        [17, 26, 15, 17, 10,  0, 32, 53],
        [57, 58,  6,  1, 61, 47, 58, 46]])

In [39]:
y

tensor([[ 1, 46, 47, 57,  1, 50, 53, 60],
        [58, 46, 43, 56, 43,  1, 41, 39],
        [26, 15, 17, 10,  0, 32, 53,  1],
        [58,  6,  1, 61, 47, 58, 46,  0]])

In [53]:
torch.manual_seed(42)
class BigramModel(nn.Module):
    def __init__(self,vocab_size):
        super().__init__()
        self.embedding_table = nn.Embedding(vocab_size,vocab_size)
    def forward(self,idx,targets=None):
        logits = self.embedding_table(idx)
        if targets!=None:
            B,T,C = logits.shape
            logits = logits.view(B*T,C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits,targets)
        else:
            loss = None
        return logits,loss

model = BigramModel(vocab_size)
out,loss = model(X,y)

In [51]:
loss

tensor(4.8865, grad_fn=<NllLossBackward0>)

In [54]:
opt = torch.optim.AdamW(model.parameters(),lr=1e-3)

In [61]:
batch_size = 32
for step in range(1000):
    xb,yb = get_batch("train")
    logits,loss = model(xb,yb)
    opt.zero_grad()
    loss.backward()
    opt.step()
print(loss.item())

2.432018280029297


In [62]:
torch.manual_seed(42)

B,T,C = 4,8,2

x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [73]:
# take average of previous tokens to transfer knowledge
xbow1 = torch.zeros((B,T,C))
for i in range(B):
    for j in range(T):
        prev = x[i,:j+1]
        xbow1[i,j] = torch.mean(prev,0)


In [83]:
# optimizing this with matrix multiplications
tril = torch.tril(torch.ones(T,T))
w = torch.zeros((T,T))
w = w.masked_fill(tril == 0,float('-inf'))
w = F.softmax(w,dim=1)
w = w / torch.sum(w,1,keepdims=True)
xbow2 = w @ x

In [84]:
torch.allclose(xbow1,xbow2)

True

In [98]:
# single head attention
tril = torch.tril(torch.ones(T,T))
head_size = 16
key = nn.Linear(C,head_size)
query = nn.Linear(C,head_size)
value = nn.Linear(C,head_size)
k = key(x)
q = query(x)
w = q @ k.transpose(-2,-1) * head_size**-0.5
w = w.masked_fill(tril == 0,float('-inf'))
w = F.softmax(w,dim=1)
w = w / torch.sum(w,1,keepdims=True)
v = value(x)
out = w @ v

In [99]:
out.shape

torch.Size([4, 8, 16])

In [101]:
k.var(),q.var(),w.var()

(tensor(0.5475, grad_fn=<VarBackward0>),
 tensor(0.4790, grad_fn=<VarBackward0>),
 tensor(0.0319, grad_fn=<VarBackward0>))